Gender Biases in Student Evaluations of Teachers - A Randomized, Online Experiment
====================================================


In [17]:
# boilerplate
%matplotlib inline
import math
import numpy as np
import pandas as pd
from numpy.random import random
import scipy as sp
from scipy import special
import matplotlib.pyplot as plt
from __future__ import division

# initialize PRNG
rs = np.random.RandomState(seed=1)

Permutation test code
============
You must install the _permute_ package to use this code. Install instructions can be found at https://github.com/statlab/permute.

In [26]:
from permute.core import corr, two_sample, permute_within_groups

Read data
=================

Some notes on the variables:
* **group** identifies the section the student was placed in.
* **gender** refers to the student's gender: 1 = male, 2 = female.
* **tagender** is the instructor's true gender: 1 = male, 0 = female.
* **taidgender** is the instructor's reported gender: 1 = male, 0 = female.

In [19]:
ratings = pd.read_csv("Macnell-RatingsData.csv")
categories = ratings.columns.values.tolist()[1:15]
ratings.head()

,group,professional,respect,caring,enthusiastic,communicate,helpful,feedback,prompt,consistent,fair,responsive,praised,knowledgeable,clear,overall,gender,age,tagender,taidgender
0,3,5,5,4,4,4,3,4,4,4,4,4,4,3,5,4,2,1990,0,1
1,3,4,4,4,4,5,5,5,5,3,4,5,5,5,5,4,1,1992,0,1
2,3,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,2,1991,0,1
3,3,5,5,5,5,5,3,5,5,5,5,3,5,5,5,5,2,1991,0,1
4,3,5,5,5,5,5,5,5,3,4,5,5,5,5,5,5,2,1992,0,1


# Analysis

### Ratings vs reported instructor gender

In [27]:
(p, t, ci, dist) = two_sample(ratings['overall'][ratings.taidgender==1], ratings['overall'][ratings.taidgender==0], \
                              stat = 't', interval = 'two-sided', keep_dist = True)
print 'Overall rating:'
print 't statistic:', np.round(t, 5)
print 'P-value (two-sided):', np.round(p, 5)
print '95% Confidence Interval for the P-value', np.round(ci, 5)
print 'Number of evaluations for male-identified instructors:', np.sum(ratings.taidgender==1)
print 'Number of evaluations for female-identified instructors:', np.sum(ratings.taidgender==0)

print ('\n\n{0:24} {1:8} {2:8}'.format('Category', 't', 'p-value'))
for col in categories:
    (p, t, ci) = two_sample(ratings[col][ratings.taidgender==1], ratings[col][ratings.taidgender==0], \
                              stat = 't', interval = 'two-sided', keep_dist = False)
    print ('{0:20} {1:8.2f} {2:8.2f}'.format(col, t, p))

Overall rating:
t statistic: 1.82159
P-value (two-sided): 0.04401
95% Confidence Interval for the P-value [ 0.04275  0.0453 ]
Number of evaluations for male-identified instructors: 23
Number of evaluations for female-identified instructors: 20


Category                 t        p-value 
professional             1.93     0.03
respect                  1.93     0.03
caring                   2.24     0.02
enthusiastic             2.14     0.03
communicate              2.06     0.03
helpful                  1.79     0.05
feedback                 1.63     0.07
prompt                   2.21     0.02
consistent               1.65     0.05
fair                     2.96     0.00
responsive               1.12     0.17
praised                  2.73     0.00
knowledgeable            1.64     0.07
clear                    1.37     0.11


### Ratings vs concordance of student and REPORTED instructor genders


In [1]:
ratings['gender_concordance'] = ( (ratings['gender']% 2)==ratings['taidgender'] )
stu_male = ratings[ratings['gender']==1]
stu_female = ratings[ratings['gender']==2]

(t, plow, pupper, pboth, sims) = corr(x = stu_male['overall'], \
                                      y = stu_male['gender_concordance'], seed = rs)
print 'Male students\n'
print 'Number of male students:', stu_male.shape[0], '\n'
print 'Correlation for overall rating:', t
print 'Upper p-value:', pupper
print 'Two-sided p-value:', pboth
print ('\n{0:15} {1:8} {2:8} {3:8}'.format('Category', 'Correlation',\
                                           'Upper p-value', 'Two-sided p-value'))
(t, plow, pupper, pboth, sims) = corr(x = stu_male['overall'], \
                                      y = stu_male['gender_concordance'], seed = rs)
print ('{0:15} {1:8.3f} {2:10.2f} {3:10.2f}'.format('Overall', t, pupper, pboth))

for col in categories:
    (t, plow, pupper, pboth, sims) = corr(x = stu_male[col], \
                                          y = stu_male['gender_concordance'], seed = rs)
    print ('{0:15} {1:8.3f} {2:10.2f} {3:10.2f}'.format(col, t, pupper, pboth))


(t, plow, pupper, pboth, sims) = corr(x = stu_female['overall'], \
                                      y = stu_female['gender_concordance'], seed = rs)
print '\nFemale students\n'
print 'Number of female students:', stu_female.shape[0], '\n'
print 'Correlation for overall rating:', t
print 'Upper p-value:', pupper
print 'Two-sided p-value:', pboth
print ('\n{0:15} {1:8} {2:8} {3:8}'.format('Category', 'Correlation', \
                                           'Upper p-value', 'Two-sided p-value'))
(t, plow, pupper, pboth, sims) = corr(x = stu_female['overall'], \
                                      y = stu_female['gender_concordance'], seed = rs)
print ('{0:15} {1:8.3f} {2:10.2f} {3:10.2f}'.format('Overall', t, pupper, pboth))

for col in categories:
    (t, plow, pupper, pboth, sims) = corr(x = stu_female[col], \
                                          y = stu_female['gender_concordance'], seed = rs)
    print ('{0:15} {1:8.3f} {2:10.2f} {3:10.2f}'.format(col, t, pupper, pboth))

NameError: name 'ratings' is not defined

### As a sanity check -- Ratings vs concordance of student and ACTUAL instructor genders

Since the students didn't know the instructors' actual gender, we hope that there is no correlation between gender concordance and ratings.

In [20]:
ratings['gender_concordance_actual'] = ( (ratings['gender']% 2)==ratings['tagender'] )
stu_male = ratings[ratings['gender']==1]
stu_female = ratings[ratings['gender']==2]

(t, plow, pupper, pboth, sims) = corr(x = stu_male['overall'], \
                                      y = stu_male['gender_concordance_actual'], seed = rs)
print 'Male students\n'
print 'Number of male students:', stu_male.shape[0]
print 'Correlation:', t
print 'Upper p-value:', pupper
print 'Two-sided p-value:', pboth

(t, plow, pupper, pboth, sims) = corr(x = stu_female['overall'], \
                                      y = stu_female['gender_concordance_actual'], seed = rs)
print '\nFemale students\n'
print 'Number of female students:', stu_female.shape[0]
print 'Correlation:', t
print 'Upper p-value:', pupper
print 'Two-sided p-value:', pboth

Male students

Number of male students: 20
Correlation: -0.0718144366713
Upper p-value: 0.6749
Two-sided p-value: 0.7227

Female students

Number of female students: 23
Correlation: 0.132379460479
Upper p-value: 0.3375
Two-sided p-value: 0.5665


### Difference in differences

We'd like to test whether the decrease in ratings due to self-identifying as a female is the same for the two instructors. We'll look at the test statistic $\left( \overline{\text{rating}}_{MM}-\overline{\text{rating}}_{MF}\right) - \left( \overline{\text{rating}}_{FM}-\overline{\text{rating}}_{FF}\right)$, where the first subscript refers to the instructor's actual gender and the second subscript refers to their reported gender. Under the null hypothesis of no difference, this statistic is $0$.

In [36]:
weights = np.ones(len(ratings.taidgender))
for g in ratings.group.unique():
    gg = np.array(ratings.group == g)
    if (ratings.tagender[gg]==ratings.taidgender[gg]).all():
        weights[gg] = 1/sum(gg)
    else:
        weights[gg] = -1/sum(gg)
tst = lambda ratings: (ratings*weights).sum()
obs = tst(ratings['overall'])

B = 100
dist = np.empty(B)
for b in range(B):
    perm = rs.permutation(ratings['overall'])
    dist[b] = tst(perm)

print ('{0:15} {1:8} {2:8}'.format('Category', 'Test Statistic', \
                                           'Two-sided p-value'))
print ('{0:15} {1:8.3f} {2:10.2f}'.format('Overall', obs, np.mean(abs(dist)>=abs(obs))))

for col in categories:
    obs = tst(ratings[col])
    dist = np.empty(B)
    for b in range(B):
        perm = rs.permutation(ratings['overall'])
        dist[b] = tst(perm)
    print ('{0:15} {1:8.3f} {2:10.2f}'.format(col, obs, np.mean(abs(dist)>=abs(obs))))

Category        Test Statistic Two-sided p-value
Overall           -0.458       0.47
professional      -0.155       0.89
respect           -0.155       0.85
caring            -0.201       0.79
enthusiastic       0.091       0.90
communicate       -0.492       0.46
helpful           -0.716       0.25
feedback          -0.633       0.37
prompt             0.364       0.55
consistent         0.292       0.64
fair               0.057       0.92
responsive        -0.455       0.52
praised            0.121       0.81
knowledgeable      0.405       0.54
clear             -1.053       0.09


## References

MacNell, L., Driscoll, A., and Hunt, A.N. (2014), "What’s in a Name: Exposing Gender Bias in Student Ratings of Teaching," Innovative Higher Education, 1-13.